#引入模組

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

#加載數據

##train_data

In [ ]:
ccba = pd.read_csv("public_train_x_ccba_full_hashed.csv")
cdtx = pd.read_csv("public_train_x_cdtx0001_full_hashed.csv")
custinfo = pd.read_csv("public_train_x_custinfo_full_hashed.csv")
dp = pd.read_csv("public_train_x_dp_full_hashed.csv")
remit = pd.read_csv("public_train_x_remit1_full_hashed.csv")
train_x = pd.read_csv("train_x_alert_date.csv")
train_y = pd.read_csv("train_y_answer.csv")

In [ ]:
#方法2 才須執行
dp = dp.drop(columns = ["tx_type", "info_asset_code", "fiscTxId", "txbranch", "cross_bank", "ATM"])
cdtx = cdtx.drop(columns = ["country", "cur_type"])
remit = remit.drop(columns = ["trans_no"])
dp.tx_amt = dp.tx_amt*dp.exchg_rate
dp = dp.drop(columns = ["exchg_rate"])

In [ ]:
dp.tx_date = dp.tx_date + dp.tx_time/24
dp = dp.drop(columns = ["tx_time"])
dp = dp.replace("CR",1)
dp = dp.replace("DB",0)

In [ ]:
a = pd.merge(train_x, custinfo, on = "alert_key")
data_info = a.iloc[:, :3]

In [ ]:
#整理數據
data_total = {}
for i in tqdm(range(len(data_info))):
  data_total[data_info["alert_key"][i]] = {}
  data_total[data_info["alert_key"][i]]["ccba"] = ccba[(ccba["cust_id"] == data_info["cust_id"][i]) & ((data_info["date"][i] - 30) <= ccba["byymm"]) & (ccba["byymm"] <= (data_info["date"][i]))].drop(columns = ["cust_id", "byymm"])
  data_total[data_info["alert_key"][i]]["cdtx"] = cdtx[(cdtx["cust_id"] == data_info["cust_id"][i]) & ((data_info["date"][i] - 30) <= cdtx["date"]) & (cdtx["date"] <= (data_info["date"][i]))].drop(columns = ["cust_id", "date"])
  data_total[data_info["alert_key"][i]]["dp"] = dp[(dp["cust_id"] == data_info["cust_id"][i]) & ((data_info["date"][i] - 30) <= dp["tx_date"]) & (dp["tx_date"] <= (data_info["date"][i]))].drop(columns = ["cust_id", "tx_date"])
  data_total[data_info["alert_key"][i]]["remit"] = remit[(remit["cust_id"] == data_info["cust_id"][i]) & ((data_info["date"][i] - 30) <= remit["trans_date"]) & (remit["trans_date"] <= (data_info["date"][i]))].drop(columns = ["cust_id", "trans_date"])

100%|██████████| 23906/23906 [1:17:14<00:00,  5.16it/s]


In [ ]:
#統計數據
data = {}
for i in tqdm(range(len(data_total))):
  data[data_info["alert_key"][i]] = {}
  data[data_info["alert_key"][i]]["ccba"] = {"frequency":len(data_total[data_info["alert_key"][i]]["ccba"]), "sum":data_total[data_info["alert_key"][i]]["ccba"].sum()}
  data[data_info["alert_key"][i]]["cdtx"] = {"frequency":len(data_total[data_info["alert_key"][i]]["cdtx"]), "sum":data_total[data_info["alert_key"][i]]["cdtx"].sum()}
  data[data_info["alert_key"][i]]["dp"] = {"frequency":len(data_total[data_info["alert_key"][i]]["dp"]), "sum":data_total[data_info["alert_key"][i]]["dp"].sum()}
  data[data_info["alert_key"][i]]["remit"] = {"frequency":len(data_total[data_info["alert_key"][i]]["remit"]), "sum":data_total[data_info["alert_key"][i]]["remit"].sum()}

100%|██████████| 23906/23906 [00:51<00:00, 462.73it/s]


In [ ]:
#產生data
data_x_final = pd.DataFrame()
for key0 in tqdm(data.keys()):  #資料數(alert_key)
  temp = {}
  for key1 in data[key0].keys():  #每筆資料的內容(ccba、cdtx、dp、remit)
    for key2 in data[key0][key1].keys():  #每筆資料的內容(frequency、mean、sum)
      temp["alert_key"] = key0
      if key2 == "frequency":
        temp[f"{key1}_{key2}"] = data[key0][key1][key2]
      else:
        for iii in range(len(data[key0][key1][key2])):  #每筆資料的內容(lupay、byymm...)
          temp[f"{key1}_{key2}_{data[key0][key1][key2].keys()[iii]}"] = data[key0][key1][key2][iii]
  data_x_final = data_x_final.append(temp, ignore_index = True)

100%|██████████| 23906/23906 [00:33<00:00, 717.47it/s]


In [ ]:
for i in range(len(data_x_final)):
  if data_x_final.dp_sum_debit_credit[i] / data_x_final.dp_frequency[i] >= 0.5:
    data_x_final.dp_sum_debit_credit[i] = 1
  elif data_x_final.dp_sum_debit_credit[i] / data_x_final.dp_frequency[i] < 0.5:
    data_x_final.dp_sum_debit_credit[i] = 0

<ipython-input-375-8e7df86dd75d>:2: RuntimeWarning: invalid value encountered in double_scalars
  if data_x_final.dp_sum_debit_credit[i] / data_x_final.dp_frequency[i] >= 0.5:
<ipython-input-375-8e7df86dd75d>:4: RuntimeWarning: invalid value encountered in double_scalars
  elif data_x_final.dp_sum_debit_credit[i] / data_x_final.dp_frequency[i] < 0.5:


In [ ]:
data_x_final

,alert_key,ccba_frequency,ccba_sum_lupay,ccba_sum_cycam,ccba_sum_usgam,ccba_sum_clamt,ccba_sum_csamt,ccba_sum_inamt,ccba_sum_cucsm,ccba_sum_cucah,...,dp_sum_tx_amt,dp_sum_exchg_rate,dp_sum_info_asset_code,dp_sum_fiscTxId,dp_sum_txbranch,dp_sum_cross_bank,dp_sum_ATM,remit_frequency,remit_sum_trans_no,remit_sum_trade_amount_usd
0,171189.0,1.0,85428.0,301224.0,154122.0,0.0,0.0,0.0,151434.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,171202.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,171599.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,171737.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1333.0,1.000,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,171142.0,1.0,12565.0,150744.0,82748.0,0.0,0.0,12477.0,12477.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,352132.0,1.0,213184.0,306146.0,200625.0,0.0,0.0,0.0,186552.0,0.0,...,9664662.0,610.222,874.0,16.0,282.0,2.0,44.0,0.0,0.0,0.0
23902,352125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5289794.0,289.550,244.0,4.0,0.0,0.0,11.0,0.0,0.0,0.0
23903,352080.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2095631.0,59.130,65.0,20.0,94.0,0.0,3.0,2.0,8.0,6602.0
23904,352075.0,1.0,0.0,199106.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3884817.0,38.000,494.0,542.0,1258.0,0.0,38.0,0.0,0.0,0.0


In [ ]:
data_final = pd.merge(data_x_final, train_y, on = ["alert_key"])
data_final

,alert_key,ccba_frequency,ccba_sum_lupay,ccba_sum_cycam,ccba_sum_usgam,ccba_sum_clamt,ccba_sum_csamt,ccba_sum_inamt,ccba_sum_cucsm,ccba_sum_cucah,...,dp_sum_exchg_rate,dp_sum_info_asset_code,dp_sum_fiscTxId,dp_sum_txbranch,dp_sum_cross_bank,dp_sum_ATM,remit_frequency,remit_sum_trans_no,remit_sum_trade_amount_usd,sar_flag
0,171189.0,1.0,85428.0,301224.0,154122.0,0.0,0.0,0.0,151434.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,171202.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,171599.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,171737.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,171142.0,1.0,12565.0,150744.0,82748.0,0.0,0.0,12477.0,12477.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,352132.0,1.0,213184.0,306146.0,200625.0,0.0,0.0,0.0,186552.0,0.0,...,610.222,874.0,16.0,282.0,2.0,44.0,0.0,0.0,0.0,0
23902,352125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,289.550,244.0,4.0,0.0,0.0,11.0,0.0,0.0,0.0,0
23903,352080.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,59.130,65.0,20.0,94.0,0.0,3.0,2.0,8.0,6602.0,0
23904,352075.0,1.0,0.0,199106.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.000,494.0,542.0,1258.0,0.0,38.0,0.0,0.0,0.0,0


In [ ]:
data_final.to_csv("train_data_final.csv")

In [ ]:
train_data_final = pd.read_csv("train_data_final.csv",index_col=False).drop(columns = ["Unnamed: 0"])
train_final = pd.merge(train_data_final, custinfo, on = ["alert_key"]).drop(columns = ["cust_id"])

In [ ]:
train_final

,alert_key,ccba_frequency,ccba_sum_lupay,ccba_sum_cycam,ccba_sum_usgam,ccba_sum_clamt,ccba_sum_csamt,ccba_sum_inamt,ccba_sum_cucsm,ccba_sum_cucah,...,dp_frequency,dp_sum_debit_credit,dp_sum_tx_amt,remit_frequency,remit_sum_trade_amount_usd,sar_flag,risk_rank,occupation_code,total_asset,AGE
0,171189.0,1.0,85428.0,301224.0,154122.0,0.0,0.0,0.0,151434.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0,1,17.0,375576.0,4
1,171202.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0,3,12.0,2717416.0,2
2,171599.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0,1,12.0,326517.0,4
3,171737.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.333000e+03,0.0,0.0,0,3,14.0,1014759.0,4
4,171142.0,1.0,12565.0,150744.0,82748.0,0.0,0.0,12477.0,12477.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0,3,12.0,241719.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,352132.0,1.0,213184.0,306146.0,200625.0,0.0,0.0,0.0,186552.0,0.0,...,52.0,0.0,1.134563e+08,0.0,0.0,0,1,19.0,3218731.0,3
23902,352125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,0.0,2.726706e+07,0.0,0.0,0,3,19.0,928963.0,3
23903,352080.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,1.635970e+07,2.0,6602.0,0,2,9.0,69080.0,6
23904,352075.0,1.0,0.0,199106.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.0,1.0,3.884817e+06,0.0,0.0,0,3,3.0,262604.0,6


In [ ]:
cols = list(train_final.columns)
train_final = train_final[cols[0:17] + cols[18:len(cols)] + [cols[17]]]
train_final = train_final.fillna(-1)
train_final

,alert_key,ccba_frequency,ccba_sum_lupay,ccba_sum_cycam,ccba_sum_usgam,ccba_sum_clamt,ccba_sum_csamt,ccba_sum_inamt,ccba_sum_cucsm,ccba_sum_cucah,...,dp_frequency,dp_sum_debit_credit,dp_sum_tx_amt,remit_frequency,remit_sum_trade_amount_usd,risk_rank,occupation_code,total_asset,AGE,sar_flag
0,171189.0,1.0,85428.0,301224.0,154122.0,0.0,0.0,0.0,151434.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,1,17.0,375576.0,4,0
1,171202.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,3,12.0,2717416.0,2,0
2,171599.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,1,12.0,326517.0,4,0
3,171737.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.333000e+03,0.0,0.0,3,14.0,1014759.0,4,0
4,171142.0,1.0,12565.0,150744.0,82748.0,0.0,0.0,12477.0,12477.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,3,12.0,241719.0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,352132.0,1.0,213184.0,306146.0,200625.0,0.0,0.0,0.0,186552.0,0.0,...,52.0,0.0,1.134563e+08,0.0,0.0,1,19.0,3218731.0,3,0
23902,352125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,0.0,2.726706e+07,0.0,0.0,3,19.0,928963.0,3,0
23903,352080.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,1.635970e+07,2.0,6602.0,2,9.0,69080.0,6,0
23904,352075.0,1.0,0.0,199106.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.0,1.0,3.884817e+06,0.0,0.0,3,3.0,262604.0,6,0


##predict_data

In [ ]:
ccba_p = pd.read_csv("private_x_ccba_full_hashed.csv")
cdtx_p = pd.read_csv("private_x_cdtx0001_full_hashed.csv")
custinfo_p = pd.read_csv("private_x_custinfo_full_hashed.csv")
dp_p = pd.read_csv("private_x_dp_full_hashed.csv")
remit_p = pd.read_csv("private_x_remit1_full_hashed.csv")
public_x = pd.read_csv("public_x_alert_date.csv")
private_x = pd.read_csv("private_x_alert_date.csv")

In [ ]:
#方法2 才須執行
cdtx_p = cdtx_p.drop(columns = ["country", "cur_type"])
remit_p = remit_p.drop(columns = ["trans_no"])
dp_p = dp_p.drop(columns = ["tx_type", "info_asset_code", "fiscTxId", "txbranch", "cross_bank", "ATM"])
dp_p.tx_amt = dp_p.tx_amt*dp_p.exchg_rate
dp_p = dp_p.drop(columns = ["exchg_rate"])

In [ ]:
dp_p.tx_date = dp_p.tx_date + dp_p.tx_time/24
dp_p = dp_p.drop(columns = ["tx_time"])
dp_p = dp_p.replace("CR",1)
dp_p = dp_p.replace("DB",0)

In [ ]:
predict_data = pd.concat([public_x, private_x], axis = 0)
custinfo_all = pd.concat([custinfo, custinfo_p], axis = 0)
ccba_all = pd.concat([ccba, ccba_p], axis = 0)
cdtx_all = pd.concat([cdtx, cdtx_p], axis = 0)
dp_all = pd.concat([dp, dp_p], axis = 0)
remit_all = pd.concat([remit, remit_p], axis = 0)

In [ ]:
a = pd.merge(predict_data, custinfo_all, on = "alert_key")
data_info_p = a.iloc[:, :3]

In [ ]:
#整理數據
data_total_p = {}
for i in tqdm(range(len(data_info_p))):
  data_total_p[data_info_p["alert_key"][i]] = {}
  data_total_p[data_info_p["alert_key"][i]]["ccba"] = ccba_p[(ccba_p["cust_id"] == data_info_p["cust_id"][i]) & ((data_info_p["date"][i] - 30) <= ccba_p["byymm"]) & (ccba_p["byymm"] <= (data_info_p["date"][i]))].drop(columns = ["cust_id", "byymm"])
  data_total_p[data_info_p["alert_key"][i]]["cdtx"] = cdtx_p[(cdtx_p["cust_id"] == data_info_p["cust_id"][i]) & ((data_info_p["date"][i] - 30) <= cdtx_p["date"]) & (cdtx_p["date"] <= (data_info_p["date"][i]))].drop(columns = ["cust_id", "date"])
  data_total_p[data_info_p["alert_key"][i]]["dp"] = dp_p[(dp_p["cust_id"] == data_info_p["cust_id"][i]) & ((data_info_p["date"][i] - 30) <= dp_p["tx_date"]) & (dp_p["tx_date"] <= (data_info_p["date"][i]))].drop(columns = ["cust_id", "tx_date"])
  data_total_p[data_info_p["alert_key"][i]]["remit"] = remit_p[(remit_p["cust_id"] == data_info_p["cust_id"][i]) & ((data_info_p["date"][i] - 30) <= remit_p["trans_date"]) & (remit_p["trans_date"] <= (data_info_p["date"][i]))].drop(columns = ["cust_id", "trans_date"])

100%|██████████| 3850/3850 [00:37<00:00, 103.47it/s]


In [ ]:
#統計數據
data_p = {}
for i in tqdm(range(len(data_total_p))):
  data_p[data_info_p["alert_key"][i]] = {}
  data_p[data_info_p["alert_key"][i]]["ccba"] = {"frequency":len(data_total_p[data_info_p["alert_key"][i]]["ccba"]), "sum":data_total_p[data_info_p["alert_key"][i]]["ccba"].sum()}
  data_p[data_info_p["alert_key"][i]]["cdtx"] = {"frequency":len(data_total_p[data_info_p["alert_key"][i]]["cdtx"]), "sum":data_total_p[data_info_p["alert_key"][i]]["cdtx"].sum()}
  data_p[data_info_p["alert_key"][i]]["dp"] = {"frequency":len(data_total_p[data_info_p["alert_key"][i]]["dp"]), "sum":data_total_p[data_info_p["alert_key"][i]]["dp"].sum()}
  data_p[data_info_p["alert_key"][i]]["remit"] = {"frequency":len(data_total_p[data_info_p["alert_key"][i]]["remit"]), "sum":data_total_p[data_info_p["alert_key"][i]]["remit"].sum()}

100%|██████████| 3850/3850 [00:08<00:00, 465.85it/s]


In [ ]:
#產生data
data_x_final_p = pd.DataFrame()
for key0 in tqdm(data_p.keys()):  #資料數(alert_key)
  temp = {}
  for key1 in data_p[key0].keys():  #每筆資料的內容(ccba、cdtx、dp、remit)
    for key2 in data_p[key0][key1].keys():  #每筆資料的內容(frequency、mean、sum)
      temp["alert_key"] = key0
      if key2 == "frequency":
        temp[f"{key1}_{key2}"] = data_p[key0][key1][key2]
      else:
        for iii in range(len(data_p[key0][key1][key2])):  #每筆資料的內容(lupay、byymm...)
          temp[f"{key1}_{key2}_{data_p[key0][key1][key2].keys()[iii]}"] = data_p[key0][key1][key2][iii]
  data_x_final_p = data_x_final_p.append(temp, ignore_index = True)

100%|██████████| 3850/3850 [00:04<00:00, 832.76it/s]


In [ ]:
for i in range(len(data_x_final_p)):
  if data_x_final_p.dp_sum_debit_credit[i] / data_x_final_p.dp_frequency[i] >= 0.5:
    data_x_final_p.dp_sum_debit_credit[i] = 1
  elif data_x_final_p.dp_sum_debit_credit[i] / data_x_final_p.dp_frequency[i] < 0.5:
    data_x_final_p.dp_sum_debit_credit[i] = 0

<ipython-input-397-70467facfba2>:2: RuntimeWarning: invalid value encountered in double_scalars
  if data_x_final_p.dp_sum_debit_credit[i] / data_x_final_p.dp_frequency[i] >= 0.5:
<ipython-input-397-70467facfba2>:4: RuntimeWarning: invalid value encountered in double_scalars
  elif data_x_final_p.dp_sum_debit_credit[i] / data_x_final_p.dp_frequency[i] < 0.5:


In [ ]:
data_x_final_p

,alert_key,ccba_frequency,ccba_sum_lupay,ccba_sum_cycam,ccba_sum_usgam,ccba_sum_clamt,ccba_sum_csamt,ccba_sum_inamt,ccba_sum_cucsm,ccba_sum_cucah,...,dp_sum_tx_amt,dp_sum_exchg_rate,dp_sum_info_asset_code,dp_sum_fiscTxId,dp_sum_txbranch,dp_sum_cross_bank,dp_sum_ATM,remit_frequency,remit_sum_trans_no,remit_sum_trade_amount_usd
0,352342.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,352866.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,352696.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,352330.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,352683.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3845,378249.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,936906.0,1141.31,968.0,40.0,4794.0,16.0,54.0,11.0,0.0,575315.0
3846,378060.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3847,378409.0,1.0,6189.0,103760.0,21264.0,0.0,0.0,998.0,15271.0,0.0,...,6681400.0,3072.97,2458.0,80.0,4928.0,26.0,145.0,19.0,0.0,964968.0
3848,377954.0,1.0,5336.0,209278.0,16057.0,0.0,0.0,4211.0,7636.0,0.0,...,8683987.0,122.60,107.0,4.0,313.0,1.0,6.0,3.0,0.0,39355.0


In [ ]:
data_x_final_p.to_csv("predict_data_final.csv")

In [ ]:
predict_data_final = pd.read_csv("predict_data_final.csv",index_col=False).drop(columns = ["Unnamed: 0"])

In [ ]:
predict_final = pd.merge(predict_data_final, custinfo_all, on = ["alert_key"]).drop(columns = ["cust_id"])
predict_final = predict_final.fillna(-1)
predict_final

,alert_key,ccba_frequency,ccba_sum_lupay,ccba_sum_cycam,ccba_sum_usgam,ccba_sum_clamt,ccba_sum_csamt,ccba_sum_inamt,ccba_sum_cucsm,ccba_sum_cucah,...,cdtx_sum_amt,dp_frequency,dp_sum_debit_credit,dp_sum_tx_amt,remit_frequency,remit_sum_trade_amount_usd,risk_rank,occupation_code,total_asset,AGE
0,352342.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1,17.0,506366.0,3
1,352866.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1,12.0,309830.0,4
2,352696.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1,2.0,444392.0,3
3,352330.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1,19.0,143930.0,4
4,352683.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,3,17.0,2112.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3845,378249.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,54.0,0.0,2.002064e+07,11.0,575315.0,1,19.0,311278.0,3
3846,378060.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1,12.0,0.0,3
3847,378409.0,1.0,6189.0,103760.0,21264.0,0.0,0.0,998.0,15271.0,0.0,...,7475.0,149.0,0.0,1.443790e+08,19.0,964968.0,1,2.0,175444.0,3
3848,377954.0,1.0,5336.0,209278.0,16057.0,0.0,0.0,4211.0,7636.0,0.0,...,2572.0,8.0,0.0,1.910845e+08,3.0,39355.0,1,17.0,1609435.0,2


#model

##Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [ ]:
X_train = train_final.iloc[:, 1:len(train_final.columns)-1]
y_train = train_final.iloc[:, len(train_final.columns)-1]
X_prediction = predict_final.iloc[:, 1:len(predict_final)]

In [ ]:
gnb.fit(X_train, y_train)

GaussianNB()

In [ ]:
proba = gnb.predict_proba(X_prediction)

In [ ]:
probability = pd.DataFrame(proba)
probability["alert_key"] = data_info_p["alert_key"]
probability
probability.to_csv("result_Bayes.csv")

##Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier()

In [ ]:
X_train = train_final.iloc[:, 1:len(train_final.columns)-1]
y_train = train_final.iloc[:, len(train_final.columns)-1]
X_prediction = predict_final.iloc[:, 1:len(predict_final)]

In [ ]:
forest.fit(X_train,y_train.values)

RandomForestClassifier()

In [ ]:
proba = forest.predict_proba(X_prediction)

In [ ]:
forest.score(X_train, y_train)

0.9999581694971974

In [ ]:
probability = pd.DataFrame(proba)
probability["alert_key"] = data_info_p["alert_key"]
probability
probability.to_csv("result_RandomForest.csv")

##XGBoost

In [ ]:
from xgboost import XGBClassifier
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.3)

In [ ]:
X_train = train_final.iloc[:, 1:len(train_final.columns)-1]
y_train = train_final.iloc[:, len(train_final.columns)-1]
X_prediction = predict_final.iloc[:, 1:len(predict_final)]

In [ ]:
xgboostModel.fit(X_train, y_train)

XGBClassifier(learning_rate=0.3)

In [ ]:
xgboostModel.score(X_train,y_train)

0.9916338994394712

In [ ]:
proba = xgboostModel.predict_proba(X_prediction)

In [ ]:
probability = pd.DataFrame(proba)
probability["alert_key"] = data_info_p["alert_key"]
probability
probability.to_csv("result_XGBoost.csv")

##lightGBM

In [ ]:
import lightgbm as lgb
lgb = lgb.LGBMClassifier(is_unbalance=True)

In [ ]:
X_train = train_final.iloc[:, 1:len(train_final.columns)-1]
y_train = train_final.iloc[:, len(train_final.columns)-1]
X_prediction = predict_final.iloc[:, 1:len(predict_final)]

In [ ]:
lgb.fit(X_train,y_train)

LGBMClassifier(is_unbalance=True)

In [ ]:
proba = lgb.predict_proba(X_prediction)

In [ ]:
probability = pd.DataFrame(proba)
probability["alert_key"] = data_info_p["alert_key"]
probability
probability.to_csv("result_lightGBM.csv")

In [ ]:
import time
while True:
  time.sleep(1)

KeyboardInterrupt: ignored